In [1]:
import numpy as np
import pandas as pd

import scipy
from scipy import stats
from sklearn import preprocessing

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

from sklearn.linear_model import LinearRegression

# spatial regression package. 
# I like the way its output looks like,
# compared to statsmodels
from pysal.model import spreg

import matplotlib
import matplotlib.pyplot as plt

In [2]:
database = "/Users/hn/Documents/01_research_data/Montgomery/"

### Question 1

In [3]:
B1_NFL1976 = pd.read_csv(database + "B1_NFL1976.csv")
# B1_NFL1976 = B1_NFL1976.rename(columns={
#     "y": "wins",
#     "x1": "rushing_yards",
#     "x2": "passing_yards",
#     "x3": "punting_avg",
#     "x4": "field_goal_pct",
#     "x5": "turnover_diff",
#     "x6": "penalty_yards",
#     "x7": "pct_rushing",
#     "x8": "opp_rushing_yard",
#     "x9": "opp_passing_yard"
# })
# B1_NFL1976.to_csv(database + 'B1_NFL1976.csv', index=False)
B1_NFL1976.head(2)

,Team,wins,rushing_yards,passing_yards,punting_avg,field_goal_pct,turnover_diff,penalty_yards,pct_rushing,opp_rushing_yard,opp_passing_yard
0,Washington,10,2113,1985,38.9,64.7,4,868,59.7,2205,1917
1,Minnesota,11,2003,2855,38.8,61.3,3,615,55.0,2096,1575


In [4]:
X = sm.add_constant(B1_NFL1976["opp_rushing_yard"])
model = sm.OLS(B1_NFL1976["wins"], X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   wins   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     31.10
Date:                Fri, 06 Feb 2026   Prob (F-statistic):           7.38e-06
Time:                        10:25:21   Log-Likelihood:                -63.123
No. Observations:                  28   AIC:                             130.2
Df Residuals:                      26   BIC:                             132.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               21.7883      2.696  

### part b.

In [5]:
# this one includes intercept on its own
model_smf = smf.ols("wins ~ opp_rushing_yard", data=B1_NFL1976)
result_smf = model_smf.fit()
anova_table = sm.stats.anova_lm(result_smf, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
opp_rushing_yard,178.092314,1.0,31.103236,0.000007
Residual,148.871972,26.0,NaN,NaN


In [6]:
result_smf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wins   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     31.10
Date:                Fri, 06 Feb 2026   Prob (F-statistic):           7.38e-06
Time:                        10:25:21   Log-Likelihood:                -63.123
No. Observations:                  28   AIC:                             130.2
Df Residuals:                      26   BIC:                             132.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           21.7883      2.696      8.081      0.000      16.246      27.330
opp_rushing_yard    -0.0070      0.001     -5.577      0.000      -0.010      -0.004
==============================================================================
Omnibus:                        2.076   Durbin-Watson:                   1.566
Prob(Omnibus):                  0.354   Jarque-Bera (JB):                1.402
Skew:                           0.305   Prob(JB):                        0.496
Kurtosis:                       2.089   Cond. No.                     1.28e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.28e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
RSS = anova_table.loc["Residual", "sum_sq"]

# Lesson:


 - ```sm.OLS``` is the array-based API --> it expects numeric matrices/arrays.

 - ```ols``` (from ```statsmodels.formula.api```) is the formula-based API --> it expects a formula string, not arrays.

And, ANOVA table uses the ols from ```statsmodels.formula.api```:
https://www.statsmodels.org/dev/generated/statsmodels.stats.anova.anova_lm.html

```result_smf.predict(X)``` from ```smf``` works just like ```result.predict(X)```. The same is true about ```conf_int()```

```get_prediction``` works like so:

```
pred = result.get_prediction([1, x0])
ci = pred.summary_frame(alpha=0.05)
```

or from ```smf```:

```
new_data = pd.DataFrame({"opp_rushing_yard": [x0]})
pred = result_smf.get_prediction(new_data)
ci = pred.summary_frame(alpha=0.05)
```



## Lets stick to ```smf```

## Working with ```smf```

Model like so

```
mpg_model = smf.ols('mpg ~ engine_displacement', data = mpg_df)
mpg_result = mpg_model.fit()
```

- It automatically adds intercept.
- ```mpg_result.summary()``` Produces summary that includes CI of coefficients at 95% sigfinicance level
- ```sm.stats.anova_lm(mpg_result, typ=2)``` creates analysis-of-variance table
- ```mpg_result.conf_int()``` shows the CI from the ```.summary()```
- ```mpg_result.conf_int(alpha=0.01)``` shows CI at 99% significance level
- RSS can be accessed in 2 ways:
  - ```(mpg_result.resid ** 2).sum()```
  - ```mpg_anova_tbl.loc["Residual", "sum_sq"]```
  
- To predict at new values of ```x``` we need a dataframe:
   - new_data = pd.DataFrame({"engine_displacement": [x0]})
   - predict_table = mpg_result.get_prediction(new_data)
   - The above result is a table with predictions, CIs and PIs.
   - ```yhat_tbl.summary_frame(alpha=0.01)```
   - Predicted values are obtained by ```yhat_tbl.predicted_mean[0]```

### Part C. done manually and using the results

In [8]:
result_smf.conf_int()

,0,1
Intercept,16.246064,27.330438
opp_rushing_yard,-0.009614,-0.004436


### manual confidence interval.

Confidence interval: $(\text{stat} - t_{\alpha/2, n-2}\text{SE}(\text{stat}), \text{stat} +  t_{\alpha/2, n-2} \text{SE}(\text{stat}))$ where stat here is the slope $\beta_1$.

Variance of $\hat \beta_1 = \frac{\sigma^2}{s_{xx}}$
where $s_{xx} = \sum(x_i - \bar x)^2$. 

**Note** In other books $s_{xx}$ might be degined differently.

Since we do not know $\sigma^2$ we estimate it by $\hat \sigma^2 = \frac{\text{RSS}}{df} = \frac{\text{RSS}}{n-2} = \text{MS}_{\text{Res}}$, and variance becomes SE and normal distribution becomes t-distribution and so on. and we end up with

$$(\hat \beta_1 - t_{\alpha/2, n-2} \text{SE}(\hat \beta_1), \hat \beta_1 + t_{\alpha/2, n-2} \text{SE}(\hat \beta_1)),$$
where $\text{SE}(\hat \beta_1) = \sqrt{\text{MS}_{Res} / S_{xx}} = \sqrt{\frac{RSS} {(n-2) \times S_{xx}}}$

In [9]:
degr_of_fr = len(B1_NFL1976)-2

x_mean = B1_NFL1976["opp_rushing_yard"].mean()
S_xx = sum((B1_NFL1976["opp_rushing_yard"] - x_mean)**2)

SE_B1 = np.sqrt(RSS / (degr_of_fr * S_xx))
slope = result.params["opp_rushing_yard"]

alpha_95 = stats.t.ppf(0.975, df=degr_of_fr)
alpha_95

[slope - alpha_95 * SE_B1, slope + alpha_95 * SE_B1] 

[-0.009614347014053563, -0.004435853537959318]

### Part (d)

We see from above that $R^2 = 0.545$

$$R^2 = \frac{\text{SS}_R}{\text{SS}_T} = \frac{\sum (\hat y_i - \bar y)^2}{\sum (y_i - \bar y) ^ 2} = 1 - \frac{\text{RSS}}{\text{SS}_T}$$

In [10]:
yhats = result.predict(X)

In [11]:
total_variability = sum((B1_NFL1976["wins"] - B1_NFL1976["wins"].mean())**2)
round(total_variability, 2)

326.96

In [12]:
remaining_variability = sum((yhats - B1_NFL1976["wins"].mean())**2)
round(remaining_variability, 2)

178.09

In [13]:
variability_explained = total_variability - remaining_variability
variability_explained_pct = 100 * (variability_explained / total_variability)
round(variability_explained_pct, 2)

45.53

In [14]:
## If we take R^2 route: R2 = 1 - RSS / SS_T:
## RSS: measure of variability after X has been considered
round(1 - result.ssr / total_variability, 4)

0.5447

### part 1e.

We are looking for CI of a $y$ at $x_0$. From Eq. 2.43 on page 31 we have

Let 

$$\text{SE}(\widehat{E[Y | x_0]}) = \sqrt{\frac{\text{RSS}}{n-2} \left(\frac{1}{n} + \frac{(x_0 - \bar x)^2}{S_{xx}} \right)}$$

then we have:

$$ E[y | x_0] \in \left(\hat \mu_{y | x_0} - t_{\alpha/2, n-2} \text{SE}, \hat \mu_{y | x_0} + t_{\alpha/2, n-2} \text{SE} \right) = \text{CI}$$

Lets do it manually and using ```stats``` way

In [15]:
x0 = 2000
pred = result.get_prediction([1, x0])
ci = pred.summary_frame(alpha=0.05)
ci

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,7.73805,0.473015,6.765753,8.710348,2.724248,12.751852


In [16]:
yhat_2000 = result.predict([1, x0])[0]
SE = np.sqrt( (RSS /degr_of_fr) * (1/(degr_of_fr+2)) + (x0 - x_mean)**2 / (S_xx)  )
alpha_SE = alpha_95 * SE

In [17]:
lower_CI_y2000 = round(yhat_2000 - alpha_SE, 3)
upper_CI_y2000 = round(yhat_2000 + alpha_SE, 3)
[lower_CI_y2000, upper_CI_y2000]

[6.801, 8.675]

#### Problem 2. Prediction interval. 

They are asking at 90\% significance level.

Again we do it in 2 ways. Recall:

Let 

$$\text{SE}(y_0 - \hat y_0) = \sqrt{\frac{\text{RSS}}{n-2} \left(1 + \frac{1}{n} + \frac{(x_0 - \bar x)^2}{S_{xx}} \right)}$$

then we have:

$$ y_0 \in \left(\hat \mu_{y | x_0} - t_{\alpha/2, n-2} \text{SE}, \hat \mu_{y | x_0} + t_{\alpha/2, n-2} \text{SE} \right) = \text{PI}$$

Lets do it manually and using ```stats``` way

In [18]:
x0 = 1800
new_data = pd.DataFrame({"opp_rushing_yard": [x0]})
pred = result_smf.get_prediction(new_data)
ci = pred.summary_frame(alpha=0.1)
ci

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,9.14307,0.597594,8.123803,10.162337,4.936392,13.349749


In [19]:
yhat_1800 = result.predict([1, x0])[0]

alpha_90 = stats.t.ppf(0.95, df=degr_of_fr)
SE = np.sqrt( (RSS /degr_of_fr) * (1 + 1/(degr_of_fr+2)) + (x0 - x_mean)**2 / (S_xx) )
alpha_SE = alpha_90 * SE

In [20]:
lower_PI_y1800 = round(yhat_1800 - alpha_SE, 3)
upper_PI_y1800 = round(yhat_1800 + alpha_SE, 3)
[lower_PI_y1800, upper_PI_y1800]

[4.98, 13.306]

In [21]:
Rocket_propellant = pd.read_csv(database + "Rocket_propellant.csv")
Rocket_propellant.head(2)

,Observation,Shear strength,Age of propellant
0,1,2158.00,15.50
1,2,1678.15,23.75


In [22]:
y = Rocket_propellant["Shear strength"]
ybar = y.mean()

x = Rocket_propellant["Age of propellant"]
xbar= x.mean()

In [23]:
sum((y - ybar) * (x - xbar))

-41132.150624999995

In [24]:
sum(y * (x - xbar))

-41132.15062500004

In [25]:
sum(ybar * (xbar - x))

4.001776687800884e-11

## Question 2.3

The text in question says $x_4$ is radial deflection of the deflected rays but the text under that table in Appendix says differently (position of focal point in north direction.)

At any rate! Lets compute $\beta_0$ and $\beta_1$ manually as a review. It was in stackPile(?) interview question. Ridiculous.

$$\beta_1 = \frac{s_{xy}}{s_{xx}} = \frac{\sum y_i (x_i - \bar x)}{\sum (x_i - \bar x)^2} =  \frac{\sum (y_i - \bar y) (x_i - \bar x)}{\sum (x_i - \bar x)^2}$$

Last equality: It turns out $\sum \bar y (x_i - \bar x) = 0$.


**Expanded version**

$$S_{xy} = \sum x_i y_i - \frac{\sum x_i \sum y_i}{n}, S_{xx} = \sum x_i^2 - \frac{\left(\sum x_i \right)^2}{n} $$

**Part a**

In [26]:
B2_SolarThEn = pd.read_csv(database + "B2_SolarThEn.csv")
B2_SolarThEn.head(2)

,total_heat_flux,insolation,focal_east,focal_south,focal_north,time_of_day
0,271.8,783.35,33.53,40.55,16.66,13.20
1,264.0,748.45,36.50,36.19,16.46,14.11


In [27]:
SolarTh_model_smf = smf.ols("total_heat_flux ~ focal_north", data=B2_SolarThEn)
SolarTh_result_smf = SolarTh_model_smf.fit()
SolarTh_result_smf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        total_heat_flux   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.710
Method:                 Least Squares   F-statistic:                     69.61
Date:                Fri, 06 Feb 2026   Prob (F-statistic):           5.94e-09
Time:                        10:25:21   Log-Likelihood:                -112.96
No. Observations:                  29   AIC:                             229.9
Df Residuals:                      27   BIC:                             232.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     607.1033     42.906     14.150      0.000     519.067     695.139
focal_north   -21.4025      2.565     -8.343      0.000     -26.666     -16.139
==============================================================================
Omnibus:                        0.520   Durbin-Watson:                   0.963
Prob(Omnibus):                  0.771   Jarque-Bera (JB):                0.362
Skew:                          -0.260   Prob(JB):                        0.834
Kurtosis:                       2.828   Cond. No.                         315.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
y = B2_SolarThEn['total_heat_flux']
x = B2_SolarThEn['focal_north']
s_xy = sum(y*x) - (sum(y) * sum(x) / len(y))
s_xx = sum(x*x) - sum(x) * sum(x) / len(y)

beta_1 = s_xy / s_xx
beta_0 = y.mean() - beta_1 * x.mean()

[round(beta_0, 3), round(beta_1, 3)]

[607.103, -21.402]

**part b: ANOVA**

In [29]:
sm.stats.anova_lm(SolarTh_result_smf, typ=2)

,sum_sq,df,F,PR(>F)
focal_north,10578.684573,1.0,69.609437,5.935009e-09
Residual,4103.243703,27.0,NaN,NaN


**Part C: 99% CI on slope**

In [30]:
SolarTh_result_smf.conf_int()

,0,1
Intercept,519.067251,695.139279
focal_north,-26.665915,-16.139001


In [31]:
df = len(y) - 2

# RSS: Reminder: RSS = sm.stats.anova_lm(SolarTh_result_smf, typ=2).loc["Residual", "sum_sq"]
rss = (SolarTh_result_smf.resid ** 2).sum()
MS_res = rss / df

SE = np.sqrt(MS_res / s_xx)
alpha_99 = stats.t.ppf(0.995, df=degr_of_fr)

[beta_1 - alpha_99 * SE, beta_1 + alpha_99 * SE]

[-28.530554152381345, -14.274362426432614]

In [32]:
# to double check with the package output.
df = len(y) - 2

# RSS: Reminder: RSS = sm.stats.anova_lm(SolarTh_result_smf, typ=2).loc["Residual", "sum_sq"]
rss = (SolarTh_result_smf.resid ** 2).sum()
MS_res = rss / df

SE = np.sqrt(MS_res / s_xx)
alpha_95 = stats.t.ppf(0.975, df=degr_of_fr)

[beta_1 - alpha_95 * SE, beta_1 + alpha_95 * SE]

[-26.6754040029989, -16.12951257581506]

**Part D: $R^2$**

$$R^2 = \frac{\text{SS}_R}{\text{SS}_T} = \frac{\sum (\hat y_i - \bar y)}{\sum (y_i - \bar y)} = 1 - \frac{\text{RSS}}{\text{SS}_T}$$

In [33]:
print(f"Python R^2 = {SolarTh_result_smf.rsquared:.3f}")

SS_T = sum((y - y.mean())**2)
print(f"Manual R^2 = {1 - rss/SS_T:.3f}")

Python R^2 = 0.721
Manual R^2 = 0.721


**Part E: 95% CI on mean heat flux when radial deflection is 16.5**

Once again

We are looking for CI of a $y$ at $x_0$. From Eq. 2.43 on page 31 we have

Let 

$$\text{SE} = \sqrt{\frac{\text{RSS}}{n-2} \left(\frac{1}{n} + \frac{(x_0 - \bar x)^2}{S_{xx}} \right)}$$

then we have:

$$ E[y | x_0] \in \left(\hat \mu_{y | x_0} - t_{\alpha/2, n-2} \text{SE}, \hat \mu_{y | x_0} + t_{\alpha/2, n-2} \text{SE} \right) = \text{CI}$$

Lets do it manually and using ```stats``` way

In [34]:
x0 = 16.5
new_data = pd.DataFrame({"focal_north": [x0]})
yhat_165 = SolarTh_result_smf.get_prediction(new_data)
ci = yhat_165.summary_frame(alpha=0.05)
ci

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,253.962704,2.347149,249.146752,258.778655,228.21398,279.711427


In [36]:
alpha_95 = stats.t.ppf(0.975, df=df)
SE = np.sqrt(MS_res * (1/df + (x0 - x.mean())/s_xx))

alpha_95_SE = alpha_95 * SE

yhat_165 = ci['mean'][0]

lower_CI_y165 = round(yhat_165 - alpha_95_SE, 3)
upper_CI_y165 = round(yhat_165 + alpha_95_SE, 3)
[lower_CI_y165, upper_CI_y165]

[249.708, 258.217]

## Question 2.4

In [46]:
mpg_df = pd.read_csv(database + "TableB3_gasoline_mileage.csv")

mpg_df = mpg_df.rename(columns={"y" : "mpg", 
                                'x1': "engine_displacement",
                                'x2': "hp", 
                                'x3': "torque", 
                                'x4': "compression_ratio", 
                                'x5': "rear_axle_ratio",
                                'x6': "carburetor",
                                'x7': "num_trans_speeds",  
                                'x8': "length", 
                                'x9': "width", 
                                'x10': "weight", 
                                'x11': "transmission_type"
                               }
                      )

mpg_df.head(2)

,mpg,engine_displacement,hp,torque,compression_ratio,rear_axle_ratio,carburetor,num_trans_speeds,length,width,weight,transmission_type
0,18.9,350.0,165,260.0,8.0,2.56,4,3,200.3,69.9,3910,1
1,17.0,350.0,170,275.0,8.5,2.56,4,3,199.6,72.9,3860,1


### Part a

In [52]:
mpg_model = smf.ols('mpg ~ engine_displacement', data = mpg_df)
mpg_result = mpg_model.fit()
mpg_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.772
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     101.7
Date:                Fri, 06 Feb 2026   Prob (F-statistic):           3.74e-11
Time:                        10:53:27   Log-Likelihood:                -80.215
No. Observations:                  32   AIC:                             164.4
Df Residuals:                      30   BIC:                             167.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              33.7227      1.444     23.355      0.000      30.774      36.672
engine_displacement    -0.0474      0.005    -10.086      0.000      -0.057      -0.038
==============================================================================
Omnibus:                        0.194   Durbin-Watson:                   1.677
Prob(Omnibus):                  0.908   Jarque-Bera (JB):                0.135
Skew:                           0.135   Prob(JB):                        0.935
Kurtosis:                       2.831   Cond. No.                         820.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Part b

In [59]:
mpg_anova_tbl = sm.stats.anova_lm(mpg_result, typ=2)
mpg_anova_tbl

,sum_sq,df,F,PR(>F)
engine_displacement,955.719710,1.0,101.735668,3.743041e-11
Residual,281.824378,30.0,NaN,NaN


In [62]:
# RSS two ways:
rss = (mpg_result.resid ** 2).sum()
print (round(rss, 2))

mpg_anova_tbl.loc["Residual", "sum_sq"].round(2)

281.82


281.82

#### 99% CI on slope

Recall:

 - $\text{SE}(\hat \beta_1) = \sqrt{\text{MS}_{Res} / S_{xx}} = \sqrt{\frac{RSS} {(n-2) \times S_{xx}}}$
 - SE for mean of $y$ at a given $x_0$: $\text{SE}(\widehat{E[Y | x_0]}) = \sqrt{\frac{\text{RSS}}{n-2} \left(\frac{1}{n} + \frac{(x_0 - \bar x)^2}{S_{xx}} \right)}$
 - SE for prediction interval: $\text{SE}(y_0 - \hat y_0) = \sqrt{\frac{\text{RSS}}{n-2} \left(1 + \frac{1}{n} + \frac{(x_0 - \bar x)^2}{S_{xx}} \right)}$



In [71]:
# The followins is 95% CIs
mpg_result.conf_int()

,0,1
Intercept,30.773834,36.67152
engine_displacement,-0.056949,-0.03777


In [77]:
mpg_result.conf_int(alpha=0.01)

,0,1
Intercept,29.751950,37.693403
engine_displacement,-0.060272,-0.034447


In [79]:
####
#### Manual
####
degr_of_fr = len(mpg_df) - 2
alpha_99 = stats.t.ppf(0.995, df=degr_of_fr)

S_xx = sum((mpg_df["engine_displacement"] - mpg_df["engine_displacement"].mean())**2)
SE_slope = np.sqrt(rss / (degr_of_fr * S_xx))
SE_slope

slope = mpg_result.params['engine_displacement']
low_CI = slope - alpha_99 * SE_slope
hi_CI = slope + alpha_99 * SE_slope

[round(low_CI, 4), round(hi_CI, 4)]

[-0.0603, -0.0344]

### Part C. 

What percentage of total variability in MPG is accounted for by linear relatinship with engine displacement?

I am not sure if this wording means they want $R^2$ or I need to compute total variability, the variability after regression as a percentage of total variability.

In [102]:
SS_T = sum((mpg_df["mpg"] - mpg_df["mpg"].mean())**2)
SS_R = sum((mpg_result.predict() - mpg_df["mpg"].mean()) ** 2)
print (round(SS_R/SS_T, 3))
print (mpg_result.rsquared.round(3))

0.772
0.772


### Part d

95% confidence interval on prediction $x_0 = 275$

In [120]:
x0 = 275
new_data = pd.DataFrame({"engine_displacement": [x0]})
yhat_tbl = mpg_result.get_prediction(new_data)
ci = yhat_tbl.summary_frame(alpha=0.05)
ci

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,20.698793,0.543867,19.588069,21.809517,14.341472,27.056114


In [121]:
yhat = yhat_tbl.predicted_mean[0]

In [123]:
x_mean = mpg_df["engine_displacement"].mean()
alpha_95 = stats.t.ppf(0.975, df=degr_of_fr)

MS_res = rss / degr_of_fr
SE_CI_pred = np.sqrt(MS_res * (1/len(mpg_df) + ( (x0 - x_mean)**2/S_xx )))

low_CI = yhat - alpha_95*SE_CI_pred
hi_CI = yhat + alpha_95*SE_CI_pred

[round(low_CI, 3), round(hi_CI, 3)]

[19.588, 21.81]

## Recall: Summary of working with ```smf```

Model like so

```
mpg_model = smf.ols('mpg ~ engine_displacement', data = mpg_df)
mpg_result = mpg_model.fit()
```

- It automatically adds intercept.
- ```mpg_result.summary()``` Produces summary that includes CI of coefficients at 95% sigfinicance level
- ```sm.stats.anova_lm(mpg_result, typ=2)``` creates analysis-of-variance table
- ```mpg_result.conf_int()``` shows the CI from the ```.summary()```
- ```mpg_result.conf_int(alpha=0.01)``` shows CI at 99% significance level
- RSS can be accessed in 2 ways:
  - ```(mpg_result.resid ** 2).sum()```
  - ```mpg_anova_tbl.loc["Residual", "sum_sq"]```
  
- To predict at new values of ```x``` we need a dataframe:
   - ```new_data = pd.DataFrame({"engine_displacement": [x0]})```
   - ```predict_table = mpg_result.get_prediction(new_data)```
   - The above result is a table with predictions, CIs and PIs.
   - ```yhat_tbl.summary_frame(alpha=0.01)```
   - Predicted values are obtained by ```yhat_tbl.predicted_mean[0]```

In [141]:
x0 = [275, 300]
new_data = pd.DataFrame({"engine_displacement": x0})
yhat_tbl = mpg_result.get_prediction(new_data)
print (yhat_tbl.predicted_mean)
yhat_tbl.summary_frame()

[20.69879276 19.51480331]


,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,20.698793,0.543867,19.588069,21.809517,14.341472,27.056114
1,19.514803,0.546350,18.399007,20.630600,13.156594,25.873013


## Part e and f 

are similar to earlier problem. Do them when you like